# Nested 5-Fold Cross Validation For Random Forest On Contextual Features

In [ ]:
import numpy as np
import pandas as pd
import xlrd as xl
from pandas import ExcelWriter
from pandas import ExcelFile
import pprint
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import re
import pickle
from operator import itemgetter
import time, datetime
from functools import partial, update_wrapper
from openpyxl import load_workbook

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as Imb_Pipeline

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score, make_scorer, confusion_matrix

pp = pprint.PrettyPrinter(indent=4)

## Ignore warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
#np.warnings.filterwarnings('ignore')

#### Read from the pickled file:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

project_path = '/content/drive/My Drive/Code Documentation Project/Issue Classification/'

Mounted at /content/drive/


# Nested Cross-Validation on Random Forest:

In [ ]:
from numpy import loadtxt
import pandas as pd
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from sklearn import preprocessing

dfs = []
for fold in range(1,11):
    test_df=pd.read_csv(project_path+'/Ensemble Data/Literature/fold'+str(fold)+'.csv')
    dfs.append(test_df)


issue_df = pd.concat(dfs, ignore_index=True)


print(len(issue_df))
print(issue_df.head())
issue_df.info()


4330
                                        Text Content  Code  Full Length  len  \
0  As quoted from the offical website http://www....     5           61   61   
1  weâre hoping to entice you to contribute SWIG ...     2          124  124   
2                                                 +1     8            2    2   
3                                                  ð     8            1    1   
4                                               :+1:     8            4    4   

       tloc      cloc     tpos1     tpos2      clen      tlen  ...  aa  \
0  0.428571  0.020642  0.000223  0.999777  0.225806  0.097222  ...   3   
1  0.571429  0.022936  0.000223  0.999777  0.709677  0.305556  ...   3   
2  1.000000  0.034404  0.000267  0.999733  1.000000  0.013889  ...   3   
3  1.000000  0.043578  0.000390  0.999610  1.000000  0.013889  ...   3   
4  0.333333  0.071101  0.001454  0.998546  0.040000  0.013889  ...   3   

   begauth  has_code  first_turn  last_turn              Code (Origin

In [ ]:
from numpy import loadtxt
import pandas as pd
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(issue_df['Code'])
issue_df['Code (Original)'] = issue_df['Code']
issue_df['Code'] = label_encoder.transform(issue_df['Code'])

#number_of_class
class_id_map_df = issue_df.drop_duplicates(subset=['Code'])
total_class=len(class_id_map_df)
encoded_label_list = class_id_map_df['Code'].to_list()
original_label_list = class_id_map_df['Code (Original)'].to_list()
class_id_map = dict(zip(encoded_label_list, original_label_list))

categ = ['aa','begauth','has_code','first_turn','last_turn']
le = preprocessing.LabelEncoder()
issue_df[categ] = issue_df[categ].apply(le.fit_transform)

print(len(issue_df))


y=issue_df['Code']
X=issue_df[['len','tloc','cloc','tpos1','tpos2','clen','tlen','ppau','npau','aa','begauth','has_code','first_turn','last_turn']]

4330


In [ ]:
# To be used within GridSearch

inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

# To be used in outer CV
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

In [ ]:
#X = all_usable_data
y

0       5
1       2
2       8
3       8
4       8
       ..
4325    9
4326    1
4327    9
4328    9
4329    9
Name: Code, Length: 4330, dtype: int64

In [ ]:
X

,len,tloc,cloc,tpos1,tpos2,clen,tlen,ppau,npau,aa,begauth,has_code,first_turn,last_turn
0,61,0.428571,0.020642,0.000223,0.999777,0.225806,0.097222,0.000208,0.000369,3,0,0,0,0
1,124,0.571429,0.022936,0.000223,0.999777,0.709677,0.305556,0.000208,0.000369,3,0,0,0,0
2,2,1.000000,0.034404,0.000267,0.999733,1.000000,0.013889,0.000425,0.001127,3,0,0,0,0
3,1,1.000000,0.043578,0.000390,0.999610,1.000000,0.013889,0.000409,0.000214,3,0,0,0,0
4,4,0.333333,0.071101,0.001454,0.998546,0.040000,0.013889,0.004261,0.002176,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4325,84,0.500000,0.839623,0.846677,0.153323,1.000000,0.100719,0.177996,0.099331,3,1,0,0,0
4326,13,0.571429,0.889937,0.925800,0.074200,0.086957,0.014388,0.145713,0.001998,3,0,1,0,0
4327,169,0.833333,0.946541,0.928984,0.071016,1.000000,0.215827,0.001770,0.003939,3,0,0,0,0
4328,61,0.500000,0.952830,0.929304,0.070696,0.909091,0.071942,0.003939,0.000084,3,1,0,0,0


## Define Pipelines:

1. Pipeline1: class_weight=balanced

In [ ]:
pipeline1 = Pipeline([
    ('clf', RandomForestClassifier(class_weight='balanced')),
])

parameters = {
    'clf__n_estimators': (10, 50),
    'clf__min_samples_split': (2, 5),
}

## Nested Cross Validation Useing GridSearch

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

for fold in range(1,11):
    #print("TRAIN:", train_index, "TEST:", test_index)
    train_df=pd.DataFrame()
    dfs=[]
    print(fold)
    for fold2 in range(1,11):
      if(fold2!=fold):
        temp_df=pd.read_csv(project_path+'/Ensemble Data/Literature/fold'+str(fold2)+'.csv')
        dfs.append(temp_df)

    train_df=pd.concat(dfs, ignore_index=True)
    test_df=pd.read_csv(project_path+'/Ensemble Data/Literature/fold'+str(fold)+'.csv')

    X_train = X_train = train_df[['has_code', 'len', 'aa', 'begauth', 'first_turn', 'last_turn', 'clen', 'tlen', 'cloc', 'tloc', 'tpos1', 'tpos2', 'ppau', 'npau']]
    y_train = train_df['Code'].values

    X_test = test_df[['has_code', 'len', 'aa', 'begauth', 'first_turn', 'last_turn', 'clen','tlen', 'cloc', 'tloc', 'tpos1', 'tpos2', 'ppau', 'npau']]
    y_test = test_df['Code'].values

    print(len(y_test))
    print(len(y_train))

    clf1 = GridSearchCV(pipeline1, parameters, cv=inner_cv, scoring='f1_weighted')
    clf1.fit(X_train, y_train)
    y_pred=clf1.predict(X_test)

    true_labels = []
    pred_labels = []

    for e in y_test:
      true_labels.append(class_id_map[e])

    for e in y_pred:
      pred_labels.append(class_id_map[e])

    pred_df=pd.read_csv(project_path+'/Ensemble Data/Literature/fold'+str(fold)+'.csv')
    pred_df['RF']=y_pred
    pred_df.to_csv(project_path+'/Ensemble Data/Literature/fold'+str(fold)+'.csv', index=False)

    report= classification_report(true_labels,pred_labels, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df=report_df.round(2)
    print(report_df)
    #report_df.to_csv(str(fold)+'.csv', index = False)
    #report_df.to_csv('en_fold'+str(fold)+'.csv', index = False) #project_path+'classification report/Literature/Ensemble/
    report_df.to_csv(project_path+'classification report/Literature/RF/fold_'+str(fold)+'.csv', index = False)


1
433
3897
              precision  recall  f1-score  support
0                  0.60    0.50      0.55     6.00
1                  0.43    0.38      0.40    24.00
2                  0.33    0.11      0.17     9.00
3                  0.54    0.54      0.54    13.00
4                  0.47    0.53      0.49    38.00
5                  0.58    0.62      0.60    29.00
6                  0.56    0.77      0.65    13.00
7                  0.62    0.43      0.51    23.00
8                  0.66    0.66      0.66    79.00
9                  0.69    0.74      0.71   141.00
10                 0.00    0.00      0.00    12.00
11                 0.71    0.81      0.76    37.00
12                 0.60    0.33      0.43     9.00
accuracy           0.62    0.62      0.62     0.62
macro avg          0.52    0.49      0.50   433.00
weighted avg       0.60    0.62      0.61   433.00
2
433
3897
              precision  recall  f1-score  support
0                  0.50    0.33      0.40     6.00
1        

In [ ]:
sup=[0 for i in range(16)]
pr=[0 for i in range(16)]
rc=[0 for i in range(16)]
f1=[0 for i in range(16)]

TOTAL_FOLD=10
for fold in range(1,TOTAL_FOLD+1):
  report_df=pd.read_csv(project_path+'classification report/Literature/RF/fold_'+str(fold)+'.csv')
  for i in range(0,16):

    s=report_df['support'][i]
    sup[i]=sup[i]+report_df['support'][i]
    pr[i]=pr[i]+(s*report_df['precision'][i])
    rc[i]=rc[i]+(s*report_df['recall'][i])
    f1[i]=f1[i]+(s*report_df['f1-score'][i])

for i in range(0,16):
  pr[i]=pr[i]/sup[i]
  rc[i]=rc[i]/sup[i]
  f1[i]=f1[i]/sup[i]
  sup[i]=int(sup[i])

pr = [ round(i, 2) for i in pr ]
rc = [ round(i, 2) for i in rc ]
f1 = [ round(i, 2) for i in f1 ]

df = pd.DataFrame({'pr':pr,'rc':rc,'f1':f1,'sup':sup})
print(df)
df.to_csv(project_path+'classification report/Literature/RF/total.csv')



      pr    rc    f1   sup
0   0.51  0.48  0.48    61
1   0.49  0.49  0.48   245
2   0.26  0.15  0.19    83
3   0.49  0.38  0.41   124
4   0.61  0.64  0.62   377
5   0.60  0.57  0.58   288
6   0.60  0.75  0.66   131
7   0.71  0.69  0.69   230
8   0.64  0.61  0.62   798
9   0.71  0.74  0.73  1411
10  0.36  0.18  0.23   125
11  0.66  0.79  0.72   368
12  0.60  0.52  0.55    89
13  0.64  0.64  0.64     6
14  0.56  0.54  0.54  4330
15  0.63  0.64  0.63  4330
